---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # Your code here
    global df
    # Type of 4/20/2009; 04/20/09; 4/20/09; 4/3/09
    # can't have \b at start since the dates at start will miss out
    c1_1= df.str.extractall(r"(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b")
    c1_2 = df.str.extractall(r"(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b")
    c1_comb = pd.concat([c1_1,c1_2])
    c1_comb.reset_index(inplace = True)
    c1_comb_index = c1_comb['level_0']
    
    # Type of Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    c2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    c2.reset_index(inplace=True)
    c2_index = c2['level_0']
    
    # Type of 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 &
    # Type of Feb 2009; Sep 2009; Oct 2010
    c3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    c3.reset_index(inplace=True)
    c3_index = c3['level_0']
    
    # Type of Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #c4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')
    #c4.reset_index(inplace=True)
    #c4_index = c4['level_0']
    # No matches for c4
    
    # Type of 6/2008; 12/2009
    import numpy as np
    c5 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    c5.reset_index(inplace=True)
    c5_index = c5['level_0']
    chk=[]
    for i in c5_index:
        if not(i in c1_comb_index.values):
            chk.append(i)
    chk = np.asarray(chk)
    c5 = c5[c5['level_0'].isin(chk)]
    
    # Type of 2009; 2010
    c6_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    c6_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    c6 = pd.concat([c6_1,c6_2])
    c6.reset_index(inplace=True)

    c6_index = c6['level_0']
    chk=[]
    for i in c6_index:
        if not((i in c2_index.values) | (i in c3_index.values) | (i in c5_index.values)):
            chk.append(i)
    chk = np.asarray(chk)
    c6 = c6[c6['level_0'].isin(chk)]
    
    fn = c1_comb.level_0.values.tolist()+c2.level_0.values.tolist()+c3.level_0.values.tolist()+c5.level_0.values.tolist()+c6.level_0.values.tolist()
    fn = np.asarray(fn)
    
    # Getting the data to clean in the required format for the function as per the required specification
    c1_comb.columns=['level_0','match','month','day','year']
    c1_comb['year']=c1_comb['year'].apply(str)
    c1_comb['year']=c1_comb['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
    
    # Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)

    c2[1] = c2[1].apply(lambda x: x.replace(',',''))
    c2['day'] = c2[1].apply(lambda x:x.split(' ')[0])
    c2['year'] = c2[1].apply(lambda x:x.split(' ')[1])
    c2.columns=['level_0','match','month','day&year','day','year']
    c2.drop('day&year',axis=1,inplace=True) 

    c3.columns=['level_0','match','day','month','year']
    c3['day'] = c3['day'].replace(np.nan,-99)
    c3['day'] = c3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    import datetime as dt
    c3['month'] = c3.month.apply(lambda x: x[:3])
    c3['month'] = pd.to_datetime(c3.month, format='%b').dt.month

    c5.columns=['level_0','match','month','year']
    c5['day']=1
    #If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).

    c6.columns=['level_0','match','year']
    c6['day']=1
    c6['month']=1
    # If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
    fnl = pd.concat([c1_comb,c2,c3,c5,c6])
    fnl['date'] =pd.to_datetime(fnl['month'].apply(str)+'/'+fnl['day'].apply(str)+'/'+fnl['year'].apply(str))
    fnl = fnl.sort_values(by='level_0').set_index('level_0')

    finallist = fnl['date']
    
    ans_f = pd.Series([i[0] for i in sorted(enumerate(finallist), key=lambda x:x[1])],np.arange(500))
    return ans_f# Your answer here